In [1]:
import numpy 
import pandas
import tensorflow 
import nltk
import gensim
import os
import re

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
3.8.3


In [2]:
import numpy as np

## Step 1. 데이터 다운로드

In [3]:
import pandas as pd
data_path = os.getenv('HOME')+'/aiffel/transformer_chatbot/data'
data = pd.read_csv(data_path+'/ChatbotData.csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
len(data)

11823

In [5]:
questions = data['Q'].values.tolist()
answers = data['A'].values.tolist()

## Step 2. 데이터 정제

In [6]:
def preprocess_sentence(sentence):
    new_sen = []    
    for i in sentence:
        i = i.lower()
        i = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z A-Z?.!,0-9\\s]+", "", i)
        new_sen.append(i)
    return new_sen
print("done")

done


## Step 3. 데이터 토큰화

토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.
5. 중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!  
구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [7]:
VOCAB_SIZE = 20000

In [8]:
from konlpy.tag import Mecab

def build_corpus(src, tgt):    #소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
    tokenizer = Mecab()
     
    src_ = preprocess_sentence(src)  #데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
    tgt_ = preprocess_sentence(tgt)
    
    que_corpus = []
    ans_corpus = []
    
    for i in src_:
        que_corpus.append(tokenizer.morphs(i))   #토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
    
    for i in tgt_: 
        ans_corpus.append(tokenizer.morphs(i))
            
    
    return que_corpus, ans_corpus

print("슈슝")

슈슝


In [9]:
que_corpus, ans_corpus = build_corpus(questions, answers)

print(len(que_corpus))
print(len(ans_corpus))

11823
11823


In [10]:
print(que_corpus[:5])

[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']]


In [11]:
print(ans_corpus[:5])

[['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


## Step 4. Augmentation


In [12]:
import gensim as gs
from gensim.models.word2vec import Word2Vec

In [13]:
print(gs.__version__)

3.8.3


In [14]:
wv_model = Word2Vec.load('/aiffel/aiffel/transformer_chatbot/data/ko.bin')

ko.bin 받는데 오래걸렸다......   
1. gensim downgrade  
2. gensim.models.Word2Vec.load -> Word2Vec.load

In [15]:
wv_model.wv['하루']

array([-3.93273801e-01, -2.27360034e+00,  2.68486333e+00, -6.34223223e-01,
       -8.81711900e-01, -1.63079605e-01, -1.07164800e+00,  3.09138030e-01,
        1.43713737e+00, -1.58763659e+00,  1.33826554e+00,  4.21405911e-01,
       -3.37255388e-01,  7.10886642e-02, -2.72570872e+00,  1.79012328e-01,
       -3.89158875e-01, -3.34602863e-01, -3.85742247e-01,  1.55386358e-01,
        1.24995506e+00,  2.16714293e-01, -1.77383351e+00,  1.16920546e-01,
       -4.11482006e-01, -2.49976349e+00, -7.46464193e-01,  2.35664234e-01,
        3.88999768e-02, -1.31096995e+00,  4.92048800e-01, -2.01834226e+00,
       -4.13530856e-01, -3.06850582e-01, -1.11028278e+00, -3.10573488e-01,
       -3.82732600e-02,  3.73514503e-01, -3.37645233e-01,  6.75652444e-01,
       -2.92677104e-01, -2.80862376e-02, -1.68554747e+00, -2.23328805e+00,
        8.56433213e-01,  9.19628665e-02, -2.45685172e+00, -1.65213302e-01,
        7.11500168e-01,  5.61769366e-01,  1.11521316e+00, -5.64922690e-01,
       -3.13016742e-01, -

In [16]:
#유사한 단어 출력(코사인 유사도)
wv_model.wv.most_similar('공부')

[('가르치', 0.6655912399291992),
 ('중퇴', 0.6448310613632202),
 ('수학', 0.6283489465713501),
 ('졸업', 0.5976732969284058),
 ('한학', 0.5947862267494202),
 ('수료', 0.5928074717521667),
 ('영문학', 0.5868287086486816),
 ('전공', 0.5848740339279175),
 ('배우', 0.5693216919898987),
 ('강론', 0.5641564726829529)]

In [17]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = []
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res

In [18]:
import random
# Augmentation된 데이터 생성
augmented_que_corpus = []
augmented_ans_corpus = []

for i in range(len(que_corpus)):
    original_que = que_corpus[i]
    augmented_que = lexical_sub(original_que, wv_model)

    if augmented_que is not None:
        augmented_que_corpus.append(augmented_que)
        augmented_ans_corpus.append(ans_corpus[i])

# 원본 데이터와 augmentation된 데이터 결합
combined_que_corpus = que_corpus + augmented_que_corpus
combined_ans_corpus = ans_corpus + augmented_ans_corpus

# 데이터를 섞어서 새로운 데이터셋 생성
combined_data = list(zip(combined_que_corpus, combined_ans_corpus))
random.shuffle(combined_data)
combined_que_corpus, combined_ans_corpus = zip(*combined_data)

# 결과 확인
print(len(combined_que_corpus))
print(len(combined_ans_corpus))

/tmp/ipykernel_266/3254644294.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]


21867
21867


## Step 5. 데이터 벡터화

In [20]:
ans_corpus[1]

['위로', '해', '드립니다', '.']

In [21]:
import tensorflow as tf
# 타겟 데이터에 <start> 토큰과 <end> 토큰 추가
# 만약 ans_corpus의 각 요소가 리스트인 경우, 각 리스트를 문자열로 결합하려면 다음과 같이 하세요.
ans_corpus = ["<start>" + "".join(sentence) + "<end>" for sentence in ans_corpus]

In [22]:
ans_corpus[1]

'<start>위로해드립니다.<end>'

In [23]:
# 리스트를 문자열로 변환(???)
que_corpus = [' '.join(sentence) for sentence in que_corpus]
#ans_corpus = [' '.join(sentence) for sentence in ans_corpus]

# 소스 데이터와 타겟 데이터를 결합하여 전체 데이터 생성
full_corpus = que_corpus + ans_corpus


# 전체 데이터로부터 단어 사전 구축
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=5000,  # 모든 토큰을 포함하기 위해 None으로 설정?
    output_mode="int",
)

# 단어 사전을 학습 데이터에 적용
tokenizer.adapt(full_corpus)

# 소스 데이터와 타겟 데이터를 벡터화
enc_train = tokenizer(que_corpus)
dec_train = tokenizer(ans_corpus)


## Step 6. 훈련하기

In [26]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    # TODO: 코드 구현
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]
    
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])



    return sinusoid_table
print("슝=3")

슝=3


In [27]:
# Mask  생성하기
def generate_padding_mask(seq):
    # TODO: 구현
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    # TODO: 구현
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    # TODO: 구현
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask
print("슝=3")

슝=3


In [28]:
#Multi-Head Attention
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        # TODO: 구현
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        
    def split_heads(self, x):
        # TODO: 구현
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        # TODO: 구현
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    def call(self, Q, K, V, mask):
        # TODO: 구현
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attention_weights
print("슝=3")

슝=3


In [29]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        # TODO: 구현
        out = self.fc1(x)
        out = self.fc2(out)
        
        return out
print("슝=3")

슝=3


In [30]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        # TODO:  구현
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # TODO: 구현
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

print("슝=3")

슝=3


In [31]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):

        '''
        Masked Multi-Head Attention
        '''
        # TODO: 구현
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        # TODO: 구현
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        # TODO: 구현
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


In [32]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        # TODO: 구현
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
            
        return out, enc_attns
print("슝=3")

슝=3


In [33]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        # TODO: 구현
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
            
        return out, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [34]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        # TODO: 구현
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        # TODO: 구현
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [35]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=1,
    d_model=368,
    n_heads=8,
    d_ff=1024,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.2,
    shared_fc=True,
    shared_emb=True)

d_model=368

print("슝=3")

슝=3


In [36]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        # TODO: 구현
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
print("슝=3")

슝=3


In [37]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)
                                        
print("슝=3")

슝=3


In [38]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    # TODO: 구현
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
print("슝=3")

슝=3


In [73]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    # TODO: 구현    
    
    # train_step 함수 내에서 tgt를 NumPy 배열로 변환
    tgt = np.array(tgt)
    tgt = np.expand_dims(tgt, axis=0)  # 1차원 배열을 2차원 배열로 확장
    
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [74]:
EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0
    tqdm_bar = tqdm(total=len(enc_train))
    
    for (batch, (src, tgt)) in enumerate(zip(enc_train, dec_train)):
        loss, _, _, _ = train_step(src, tgt, transformer, optimizer)
        total_loss += loss
        tqdm_bar.update(1)
        tqdm_bar.set_description(f'Epoch {epoch + 1}, Loss: {total_loss / (batch + 1):.4f}')

    tqdm_bar.close()

    # 검증 데이터셋을 사용하여 에포크 종료 후 모델 성능 평가 가능
    total_val_loss = 0
    val_steps = 0

    for (batch, (src, tgt)) in enumerate(zip(enc_val, dec_val)):
        loss, _, _, _ = train_step(src, tgt, transformer, optimizer, training=False)
        total_val_loss += loss
        val_steps += 1

    avg_val_loss = total_val_loss / val_steps
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(enc_train):.4f}, Val Loss: {avg_val_loss:.4f}')



  0%|          | 0/11704 [01:45<?, ?it/s]


TypeError: in user code:

    /tmp/ipykernel_266/1977299913.py:13 train_step  *
        enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)
    /tmp/ipykernel_266/4201719780.py:14 generate_masks  *
        enc_mask = generate_padding_mask(src)
    /tmp/ipykernel_266/4201719780.py:4 generate_padding_mask  *
        seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/ops/math_ops.py:1864 equal
        return gen_math_ops.equal(x, y, name=name)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/ops/gen_math_ops.py:3217 equal
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/framework/op_def_library.py:527 _apply_op_helper
        raise TypeError(

    TypeError: Expected string passed to parameter 'y' of op